In [ ]:
import numpy as np
import matplotlib.pyplot as pl
from lmtslr.spec import *
from lmtslr.ifproc import *
from lmtslr.viewer.spec_viewer import *
#from beam import *
#from beam_viewer import *
from lmtslr.utils.roach_file_utils import create_roach_list, lookup_roach_files
from lmtslr.utils.ifproc_file_utils import lookup_ifproc_file

from lmtslr.ifproc.ifproc import IFProcData, IFProcCal
from lmtslr.spec.spec import SpecBankCal, SpecBankData

In [ ]:
path='lmt_data'
# path = '/media/teuben/P1-2/teuben/LMT/lmt_data'
# set all parameters to control the reduction and plot                                                         
obsnum = 82865 
# obsnum = 79273
pixel_number = 10
tsys = 250.
#list_of_pixels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#list_of_pixels = [2]                                                                                  
list_of_pixels = [10]
roach_list = create_roach_list(list_of_pixels)
use_calibration = True
baseline_order = 1
#baseline_list = [[-100,0],[70,100]]
#line_integral_list = [[5,65]]
plot_limits = [-100,100,-100,100]
#baseline_list = [[-200,-100],[50,150]]
#line_integral_list = [[-50,0]]
#baseline_list = [[-50,25],[75,150]]
#line_integral_list = [[26,66]]
#
baseline_list = [[-100,-50],[20,70]]
line_integral_list = [[-30,0]]

# R LMi
#baseline_list = [[-100,-20],[20,100]]
#line_integral_list = [[-20,20]]

# V Cam
#baseline_list = [[-25,5],[25,50]]
#line_integral_list = [[5,25]]

# IRC+60154
#baseline_list = [[40,50],[65,80]]
#line_integral_list = [[50,65]]
# NML Tau
#baseline_list = [[0,20],[60,80]]
#line_integral_list = [[25,45]]



# VY CMa
#baseline_list = [[-20,-5],[65,80]]
#line_integral_list = [[0,60]]



# R Leo
#baseline_list = [[-20,-5],[20,35]]
#line_integral_list = [[0,18]]

# MIRA
#baseline_list = [[30,45],[65,90]]
#line_integral_list = [[48,62]]

# Orion KL
#line_integral_list = [[-5,35]] 
#baseline_list = [[-20,-10],[40,50]]
#plot_limits = [-50,100,-1,5]

# look up files to match pixel list                                                                            
files,nfiles = lookup_roach_files(obsnum,roach_list,path=path+'/spectrometer')
ifproc_file = lookup_ifproc_file(obsnum,path+'/ifproc')

# @todo   why do the lookup_() functions need to know about the subdirs, why not use common path


In [ ]:
# create the spec_bank object.  This reads all the roaches in the list "files"       
I = IFProcData(ifproc_file)
S = SpecBankData(files,I,pixel_list=list_of_pixels,bank=0)

In [ ]:
# check whether to use calibration and open necessary file                                                     
if use_calibration == True:
    S.cal_flag = False
    calobsnum = S.calobsnum
    cal_files,ncalfiles = lookup_roach_files(calobsnum,roach_list,path=path+'/spectrometer')
    ifproc_cal_file = lookup_ifproc_file(calobsnum,path=path+'/ifproc')
    ICal = IFProcCal(ifproc_cal_file)
    SCal = SpecBankCal(cal_files,ICal,pixel_list=list_of_pixels)
    check_cal = SCal.test_cal(S)
    if check_cal > 0:
        print('WARNING: CAL MAY NOT BE CORRECT')

    # reduce all spectra - calibrated                                                                          
    for ipix in range(S.npix):
        S.roach[ipix].reduce_spectra(stype=1,
                                         calibrate=True,
                                         tsys_spectrum=SCal.roach[ipix].tsys_spectrum)
else:
    # reduce all spectra - uncalibrated                                                                        
    for ipix in range(S.npix):
        S.roach[ipix].reduce_spectra(stype=1,calibrate=False,tsys_no_cal=150)


In [ ]:
# set the baseline channels from velocities                                                                    
blist,nb = S.make_velocity_list(baseline_list,id='baseline')
# set the line integration channels from velocities                                                            
clist,nc = S.make_velocity_list(line_integral_list,id='line')

# creates the velocity scale for this observation
v = S.create_velocity_scale()

S.create_map_grid_data(clist,nc,blist,nb,baseline_order=0,pixel_list=list_of_pixels)

In [ ]:
SV = spec_bank_viewer()
SV.open_figure()
plot_limits = [-100,100,-100,100]
SV.plot_all_spectra(S,pixel_number,plot_limits,blist,nb)


In [ ]:
B = BeamMap(S,pix_list=[pixel_number])
B.fit_peaks_in_list(fit_circle=30)
BV = BeamMapView()
BV.print_pixel_fits(B)
print(S.ifproc.az_user,B.peak_fit_params[0,1],S.ifproc.el_user,B.peak_fit_params[0,3])
print('%s %d %6.1f %5.1f %6.1f %6.1f'%(S.ifproc.source,S.obsnum,S.ifproc.azim,S.ifproc.elev,S.ifproc.az_user+B.peak_fit_params[0,1],S.ifproc.el_user+B.peak_fit_params[0,3]))
print(' ')



In [ ]:
v = S.create_velocity_scale()
pix_index = S.find_pixel_index(pixel_number)
iii = 12
pl.plot(v,(S.map_spectra[pix_index][iii]-np.sum(S.map_spectra[pix_index][iii][S.blist])/S.nb),'o-')
pl.axis([-80,80,-1,10])

In [ ]:
plen = len(S.map_spectra[pix_index])
xlen = ylen = int(math.sqrt(plen))
points = []
for i in range(plen):
    points.append( (np.round(S.map_x[pix_index,i]), np.round(S.map_y[pix_index,i]),i)) 
c = sorted(points,key=lambda point: point[0])
d = sorted(c,key=lambda point: point[1], reverse=True)
print(d)
for i in range(plen):
    plot_index = d[i][2]
    ax = pl.subplot(xlen,ylen,plot_index+1)
    ax.plot(v,(S.map_spectra[pix_index][plot_index]-np.sum(S.map_spectra[pix_index][plot_index][S.blist])/S.nb))
